# AutoEncoder-影像壓縮

## 宣告使用套件庫

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import save_image
from google.colab.patches import cv2_imshow
import numpy as np

## 定義模型

In [3]:
# 定義Autoencoder模型
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),  # 輸入大小為28x28，維度壓縮至128
            nn.ReLU(True),
            nn.Linear(128, 64),  # 進一步壓縮維度至64
            
            nn.ReLU(True),
            nn.Linear(64, 10),  # 進一步壓縮維度至64
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()  # 通過Sigmoid函數將輸出壓縮至[0, 1]區間內
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x





## 定義訓練參數

In [4]:
# 設定訓練參數
batch_size = 128
learning_rate = 1e-3
num_epochs = 20
# 加載MNIST數據集
train_dataset = MNIST(root='./data', train=True, transform=ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 初始化模型、損失函數和優化器
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

100%|██████████| 9912422/9912422 [00:00<00:00, 105333365.20it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 33178771.25it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28978493.72it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12541493.59it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## 開始訓練


In [5]:
# 訓練模型
for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        img = img.view(img.size(0), -1)  # 將圖像展平成一維向量

        # 前向傳播
        output = model(img)
        loss = criterion(output, img)

        # 反向傳播和優化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 每個epoch打印訓練過程
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 0.0447
Epoch [2/20], Loss: 0.0347
Epoch [3/20], Loss: 0.0327
Epoch [4/20], Loss: 0.0302
Epoch [5/20], Loss: 0.0284
Epoch [6/20], Loss: 0.0276
Epoch [7/20], Loss: 0.0262
Epoch [8/20], Loss: 0.0273
Epoch [9/20], Loss: 0.0300
Epoch [10/20], Loss: 0.0263
Epoch [11/20], Loss: 0.0249
Epoch [12/20], Loss: 0.0267
Epoch [13/20], Loss: 0.0246
Epoch [14/20], Loss: 0.0246
Epoch [15/20], Loss: 0.0269
Epoch [16/20], Loss: 0.0229
Epoch [17/20], Loss: 0.0260
Epoch [18/20], Loss: 0.0239
Epoch [19/20], Loss: 0.0240
Epoch [20/20], Loss: 0.0234


## 測試模型

In [6]:
# 使用測試數據評估模

test_dataset = MNIST(root='./data', train=False, transform=ToTensor())
test_loader = DataLoader(test_dataset, batch_size=1,shuffle=False)

with torch.no_grad():
    for i,data in enumerate(test_loader):
        img, _ = data
        img = img.view(img.size(0), -1)
        output = model(img)
        print(model.encoder(img))
        o_img = img.view(img.size(0), 1, 28, 28)
        r_img = output.view(output.size(0), 1, 28, 28)
        # 儲存壓縮和重構的圖像
        save_image(o_img, './original.png')
        cv2_imshow(np.array(o_img[0][0])*255)
        save_image(r_img, './reconstructed.png')
        cv2_imshow(np.array(r_img[0][0])*255)
        if i>10:
            break
        #break  # 只顯示第一個批次的圖像

tensor([[ 0.0000,  0.0000, 24.4712, 16.7393,  0.0000, 11.4510, 10.0222, 13.2285,
          2.1319,  0.0000]])


tensor([[ 0.0000,  0.0000,  0.2608, 19.8776,  0.0000,  7.2978,  9.7529, 17.2677,
         10.2778,  0.0000]])


tensor([[ 0.0000,  0.0000, 13.8230, 20.9168,  0.0000, 14.4198,  0.6441,  8.2988,
          8.7360,  0.0000]])


tensor([[ 0.0000,  0.0000, 17.8135, 14.8048,  0.0000, 10.6588, 23.7675, 18.5612,
         18.0799,  0.0000]])


tensor([[ 0.0000,  0.0000, 15.6280, 11.9699,  0.0000,  7.6073, 14.6297,  8.1734,
          6.6508,  0.0000]])


...

tensor([[ 0.0000,  0.0000,  9.4408, 14.9779,  0.0000, 10.1896, 14.3082,  1.3421,
          7.6320,  0.0000]])


tensor([[ 0.0000,  0.0000, 19.7019, 12.1266,  0.0000,  9.5066, 15.5414, 13.3785,
          9.4633,  0.0000]])


tensor([[ 0.0000,  0.0000, 38.2711, 25.7138,  0.0000, 15.5194, 14.7493, 22.0407,
          9.8501,  0.0000]])


tensor([[ 0.0000,  0.0000, 11.1411,  7.7323,  0.0000,  7.0183, 17.6344, 16.1892,
          8.6347,  0.0000]])


tensor([[ 0.0000,  0.0000,  6.6014, 11.2236,  0.0000,  3.7586, 13.8223, 10.3781,
          8.8555,  0.0000]])


## 單獨使用解碼器將特徵向量反解為圖像

In [8]:
import numpy  as np
from google.colab.patches import cv2_imshow
# 宣告測試向量
tensor_test = torch.Tensor([[ 0.0000,  0.0000, 24.4712, 16.7393,  0.0000, 11.4510, 10.0222, 13.2285,
          2.1319,  0.0000]])
with torch.no_grad():
    # 將向量進行解碼
    output = model.decoder(tensor_test)
cv2_imshow(np.array(output.view(output.size(0), 1, 28, 28)[0][0])*255)